**Earthquake Prediction Pipeline using Transformer Models**

A comprehensive system for predicting earthquake occurrences using USGS data and deep learning.

Core Capabilities:
- Automated USGS data collection and processing
- Transformer-based sequence modeling
- Real-time prediction and evaluation
- Continuous model optimization
- Performance visualization

Process Flow:
1. Data Collection
   - Pulls earthquake data from USGS API
   - Filters by magnitude threshold
   - Organizes by date

2. Data Processing
   - Daily data segmentation
   - Feature extraction and scaling
   - Sequence creation
   - Tensor conversion

3. Model Pipeline
   - Initial training on historical data
   - Daily predictions
   - Performance evaluation
   - Model optimization
   - Checkpoint management

4. Monitoring System
   - Continuous data collection
   - Automated predictions
   - Real-time evaluation
   - Model updates
   - Performance tracking

Authors: Stephen Moore / Steven Willhelm / Lynn Yingling
Date: November 2024
Version: 3.0

In [21]:
# Required imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from datetime import datetime, timedelta
import requests
import os
import json
import glob
import time
from sklearn.preprocessing import MinMaxScaler
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [22]:
def setup_drive_directory(base_path='earthquake_data'):
    """Mount Google Drive and create necessary directories"""
    drive.mount('/content/drive')
    full_path = f'/content/drive/My Drive/{base_path}'
    if not os.path.exists(full_path):
        os.makedirs(full_path)
        print(f"Created directory: {full_path}")
    else:
        print(f"Directory already exists: {full_path}")
    return full_path

In [23]:
def fetch_earthquake_data(self, start_time=None, end_time=None, min_magnitude=2.5, use_rolling_window=False):
    """
    Fetch earthquake data from USGS API for a specific time period.

    Args:
        start_time (datetime): Start of time period
        end_time (datetime): End of time period
        min_magnitude (float): Minimum earthquake magnitude to include
        use_rolling_window (bool): If True, use rolling 24-hour window instead of calendar day

    Returns:
        pandas DataFrame with earthquake data
    """
    try:
        base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

        # Format dates for the API
        if start_time is None:
            start_time = datetime.now() - timedelta(hours=24)
        if end_time is None:
            end_time = datetime.now()

        params = {
            'format': 'geojson',
            'starttime': start_time.strftime('%Y-%m-%dT%H:%M:%S'),  # Include time
            'endtime': end_time.strftime('%Y-%m-%dT%H:%M:%S'),      # Include time
            'minmagnitude': min_magnitude,
            'orderby': 'time'
        }

        if use_rolling_window:
            print(f"Fetching data from: {start_time} to {end_time}")
        else:
            print(f"Fetching data for: {start_time.strftime('%Y-%m-%d')}")

        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()
        earthquakes = data['features']

        processed_data = []
        for quake in earthquakes:
            properties = quake['properties']
            coordinates = quake['geometry']['coordinates']

            processed_data.append({
                'time': datetime.fromtimestamp(properties['time'] / 1000),
                'magnitude': properties['mag'],
                'place': properties['place'],
                'longitude': coordinates[0],
                'latitude': coordinates[1],
                'depth': coordinates[2],
                'type': properties['type'],
                'alert': properties.get('alert', 'none'),
                'tsunami': properties['tsunami'],
                'sig': properties['sig']
            })

        df = pd.DataFrame(processed_data)

        if not df.empty:
            df['time'] = pd.to_datetime(df['time'])

            if not use_rolling_window:
                # Filter to calendar day if not using rolling window
                start_day = pd.Timestamp(start_time.strftime('%Y-%m-%d'))
                end_day = start_day + pd.Timedelta(days=1)
                df = df[
                    (df['time'] >= start_day) &
                    (df['time'] < end_day)
                ]

            if len(df) > 0:
                print("\nData Collection Summary:")
                print("-" * 30)
                print(f"Total earthquakes collected: {len(df)}")
                print(f"Date range: {df['time'].min()} to {df['time'].max()}")
                print(f"Magnitude range: {df['magnitude'].min():.1f} to {df['magnitude'].max():.1f}")
                print("-" * 30)

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

In [24]:
def fetch_training_data(self, start_date, end_date):
    """Fetch training data for specified date range"""
    df = self.fetch_earthquake_data(
        start_time=start_date,
        end_time=end_date,
        min_magnitude=2.5
    )

    if df is not None:
        # Save with date range
        filename = f'earthquake_data_{start_date.strftime("%Y%m%d")}_to_{end_date.strftime("%Y%m%d")}.csv'
        filepath = os.path.join(self.drive_path, filename)
        df.to_csv(filepath, index=False)

        return df, filepath
    return None, None

In [25]:
def fetch_new_data(self, last_timestamp):
    """Fetch only new data since last recorded timestamp"""
    df = self.fetch_earthquake_data(
        start_time=last_timestamp,
        end_time=datetime.now(),
        min_magnitude=2.5
    )

    if df is not None:
        # Filter to only new events
        new_data = df[df['time'] > last_timestamp]

        if len(new_data) > 0:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'new_data_{timestamp}.csv'
            filepath = os.path.join(self.dirs['data'], filename)
            new_data.to_csv(filepath, index=False)

            return new_data, filepath
    return None, None

In [8]:
class EarthquakeDataset(Dataset):
    """
    Custom Dataset class for handling earthquake sequence data.

    This class creates sequences of earthquake data for training the transformer model.
    Each sequence consists of 'seq_length' days of data, with the target being the
    subsequent day's earthquake parameters.

    Attributes:
        features (tensor): Input features for each earthquake event
        targets (tensor): Target values for prediction
        seq_length (int): Number of days in each sequence

    Example:
        >>> features = torch.randn(100, 5)  # 100 events with 5 features each
        >>> targets = torch.randn(100, 3)   # 3 target parameters for each event
        >>> dataset = EarthquakeDataset(features, targets, seq_length=7)
        >>> sequence, target = dataset[0]  # Get first sequence and its target
    """

    def __init__(self, features, targets, seq_length):
        """
        Initialize the dataset with features, targets, and sequence length.

        Args:
            features (torch.Tensor): Input features for each earthquake event
            targets (torch.Tensor): Target values for prediction
            seq_length (int): Number of days to include in each sequence
        """
        self.features = features
        self.targets = targets
        self.seq_length = seq_length

    def __len__(self):
        """Return the number of possible sequences in the dataset."""
        return max(0, len(self.features) - self.seq_length)

    def __getitem__(self, idx):
        """
        Get a sequence of features and its corresponding target.

        Args:
            idx (int): Index of the sequence to retrieve

        Returns:
            tuple: (feature_sequence, target) where feature_sequence is a sequence of
                  'seq_length' days of data and target is the next day's parameters
        """
        feature_seq = self.features[idx:idx + self.seq_length]
        target = self.targets[idx + self.seq_length - 1]
        return feature_seq, target

In [9]:
class TransformerPredictor(nn.Module):
    """
    Transformer-based earthquake prediction model.

    Architecture:
    [Input Events] → [Projection Layer] → [Positional Encoding] →
    [Transformer Encoder] → [Output Heads]

    Layer Dimensions:
    - Input: (batch_size, seq_length, max_earthquakes, input_dim)
    - Projection: input_dim → adjusted_dim
    - Encoder: adjusted_dim → adjusted_dim
    - Output: adjusted_dim → target_dim

    Attention Mechanism:
    - Multi-head self-attention (num_heads=4)
    - Separate attention for temporal and spatial dimensions
    - Scaled dot-product attention with dropout

    Loss Function:
    - MSE loss for parameter prediction
    - Binary cross-entropy for earthquake count
    - Combined loss with learned weights

    Components:
    1. Input Projection
      - Linear projection to match transformer dimensions
      - Handles varying input features

    2. Positional Encodings
      - Temporal encoding for sequence days
      - Spatial encoding for earthquake events
      - Learned parameters for both dimensions

    3. Transformer Encoder
      - Multiple self-attention layers
      - Feed-forward networks
      - Layer normalization
      - Dropout for regularization

    4. Prediction Heads
      - Separate heads for parameters and count
      - Non-linear activation functions
      - Confidence estimation

    Args:
        input_dim (int): Number of input features
        hidden_dim (int): Dimension of hidden layers
        num_layers (int): Number of transformer layers
        num_heads (int): Number of attention heads
        max_earthquakes (int): Maximum events per day
        min_seq_length (int): Minimum sequence length
        max_seq_length (int): Maximum sequence length
    """

    def __init__(self, input_dim, hidden_dim, num_layers, num_heads, max_earthquakes,
                 min_seq_length=1, max_seq_length=7):
        """
        Initialize the transformer model.

        Args:
            input_dim (int): Number of input features
            hidden_dim (int): Dimension of hidden layers
            num_layers (int): Number of transformer encoder layers
            num_heads (int): Number of attention heads
            max_earthquakes (int): Maximum number of earthquakes per day
            min_seq_length (int, optional): Minimum sequence length. Defaults to 1.
            max_seq_length (int, optional): Maximum sequence length. Defaults to 7.
        """
        super().__init__()

        # Adjust input dimension to be divisible by num_heads
        self.adjusted_dim = ((input_dim // num_heads) + 1) * num_heads
        self.min_seq_length = min_seq_length
        self.max_seq_length = max_seq_length

        # Initial projection to match transformer dimensions
        self.input_projection = nn.Linear(input_dim, self.adjusted_dim)

        # Positional encodings for temporal and spatial dimensions
        self.day_pos_encoding = nn.Parameter(torch.randn(1, max_seq_length, 1, self.adjusted_dim))
        self.eq_pos_encoding = nn.Parameter(torch.randn(1, 1, max_earthquakes, self.adjusted_dim))

        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.adjusted_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)

        # Prediction heads
        self.earthquake_predictor = nn.Sequential(
            nn.Linear(self.adjusted_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)  # [magnitude, latitude, longitude]
        )

        self.num_eq_predictor = nn.Sequential(
            nn.Linear(self.adjusted_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_length, max_eq, features)

        Returns:
            tuple: (earthquake_predictions, num_earthquakes) where earthquake_predictions
                  contains the predicted parameters for each potential earthquake and
                  num_earthquakes is the predicted number of earthquakes
        """
        batch_size, seq_length, max_eq, features = x.shape

        # Project input features
        x = x.view(-1, features)
        x = self.input_projection(x)
        x = x.view(batch_size, seq_length, max_eq, -1)

        # Add positional encodings
        x = x + self.day_pos_encoding[:, :seq_length]  # Temporal encoding
        x = x + self.eq_pos_encoding                   # Spatial encoding

        # Reshape for transformer
        x = x.view(batch_size, seq_length * max_eq, -1)

        # Apply transformer encoder
        encoded = self.transformer(x)

        # Reshape and generate predictions
        encoded = encoded.view(batch_size, seq_length, max_eq, -1)
        encoded = encoded[:, -1]  # Use last day's encodings

        # Generate predictions
        eq_predictions = self.earthquake_predictor(encoded)
        num_earthquakes = self.num_eq_predictor(encoded.mean(dim=1))

        return eq_predictions, num_earthquakes

In [ ]:
class EarthquakePipeline:
    """
    Earthquake prediction pipeline using transformer models and USGS data.

    This pipeline implements:
    1. Automated data collection from USGS
    2. Daily data processing and storage
    3. Model training and optimization
    4. Prediction generation and evaluation
    5. Continuous monitoring capabilities

    Configuration:
    - sequence_length: Number of days used for prediction (default: 7)
    - prediction_horizon: Days ahead to predict (default: 1)
    - feature_columns: ['magnitude', 'latitude', 'longitude', 'depth', 'sig']
    - target_columns: ['magnitude', 'latitude', 'longitude']
    - max_earthquakes: Maximum events per day (fixed at 60)

    Directory Structure:
    /earthquake_data/
    ├── data/             # Raw daily earthquake data
    │   └── YYYY-MM/     # Organized by year-month
    ├── models/          # Saved model checkpoints
    ├── predictions/     # Daily prediction outputs
    ├── plots/          # Performance visualizations
    └── evaluations/    # Evaluation metrics

    Data Persistence:
    - Daily data stored as CSV files
    - Model checkpoints saved after optimization
    - Predictions stored as dated CSV files
    - Performance metrics saved as JSON
    - Visualization plots saved as PNG

    Performance Metrics:
    - Location error (km)
    - Magnitude error
    - Precision/Recall/F1
    - False positive/negative rates
    - Prediction confidence

    Optimization Strategy:
    - Daily model updates using new data
    - Adaptive learning rate
    - Early stopping on validation loss
    - Performance-based checkpoint saving

    Example:
        >>> pipeline = EarthquakePipeline('/content/drive/My Drive/earthquake_data')
        >>> pipeline.run_pipeline(days_to_process=30, continuous=True)
    """

    def __init__(self, drive_path, seq_length=7, prediction_horizon=1):
        """
        Initialize the pipeline with directories and parameters.

        Args:
            drive_path (str): Base path in Google Drive for storing all pipeline data
            seq_length (int, optional): Number of days in each sequence. Defaults to 7.
            prediction_horizon (int, optional): Days ahead to predict. Defaults to 1.
        """
        self.drive_path = drive_path

        # Create structured directories
        self.dirs = {
            'data': os.path.join(drive_path, 'data'),
            'models': os.path.join(drive_path, 'models'),
            'predictions': os.path.join(drive_path, 'predictions'),
            'plots': os.path.join(drive_path, 'plots'),
            'evaluations': os.path.join(drive_path, 'evaluations')
        }

        # Create all directories
        for dir_path in self.dirs.values():
            os.makedirs(dir_path, exist_ok=True)

        # Initialize parameters
        self.seq_length = seq_length
        self.prediction_horizon = prediction_horizon
        self.feature_scaler = MinMaxScaler()
        self.target_scaler = MinMaxScaler()
        self.model = None
        self.feature_columns = ['magnitude', 'latitude', 'longitude', 'depth', 'sig']
        self.target_columns = ['magnitude', 'latitude', 'longitude']
        self.performance_history = []

        # Date tracking system
        self.model_dates = {
            'last_training_date': None,
            'last_optimization_date': None,
            'latest_data_date': None,
            'prediction_target_date': None
        }

        # Initialize metadata
        self.metadata_path = os.path.join(drive_path, 'pipeline_metadata.json')
        self._load_or_create_metadata()

    def _create_new_metadata(self):
        """Create new metadata structure with all required fields."""
        self.metadata = {
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'data_dates': [],  # List to store dates of processed data
            'model_versions': [],  # List to store model version information
            'predictions': [],  # List to store prediction records
            'evaluations': [],  # List to store evaluation results
            'pipeline_config': {
                'sequence_length': self.seq_length,
                'prediction_horizon': self.prediction_horizon,
                'feature_columns': self.feature_columns,
                'target_columns': self.target_columns
            }
        }
        self._save_metadata()

    def _ensure_metadata_structure(self):
        """Ensure all required fields exist in metadata."""
        required_fields = {
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'data_dates': [],
            'model_versions': [],
            'predictions': [],
            'evaluations': [],
            'pipeline_config': {
                'sequence_length': self.seq_length,
                'prediction_horizon': self.prediction_horizon,
                'feature_columns': self.feature_columns,
                'target_columns': self.target_columns
            }
        }

        # Add any missing fields
        for key, default_value in required_fields.items():
            if key not in self.metadata:
                self.metadata[key] = default_value
                print(f"Added missing metadata field: {key}")

        # Add any missing nested fields in pipeline_config
        if 'pipeline_config' in self.metadata:
            for key, value in required_fields['pipeline_config'].items():
                if key not in self.metadata['pipeline_config']:
                    self.metadata['pipeline_config'][key] = value
                    print(f"Added missing config field: {key}")

    def _save_metadata(self, verbose=False):
        """
        Save pipeline metadata to JSON file.

        Handles serialization of metadata including:
        - Pipeline configuration and status
        - Data tracking and ranges
        - Model versions and training history
        - Prediction history and performance metrics
        - File paths and timestamps
        """
        try:
            from datetime import date, datetime
            import numpy as np
            import pandas as pd

            # Create comprehensive metadata structure
            metadata = {
                'last_update': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'pipeline_info': {
                    'creation_date': self.metadata.get('creation_date'),
                    'sequence_length': self.seq_length,
                    'feature_columns': self.feature_columns,
                    'target_columns': self.target_columns
                },
                'data_range': {
                    'start': self.model_dates.get('first_data_date'),
                    'end': self.model_dates.get('latest_data_date'),
                    'total_days_processed': len(self.metadata.get('data_dates', []))
                },
                'training_info': {
                    'last_training': self.model_dates.get('last_training_date'),
                    'last_optimization': self.model_dates.get('last_optimization_date'),
                    'model_versions': self.metadata.get('model_versions', [])
                },
                'prediction_stats': self.performance_history,
                'file_paths': {
                    'data_directory': self.dirs['data'],
                    'model_directory': self.dirs['models'],
                    'predictions_directory': self.dirs['predictions'],
                    'plots_directory': self.dirs['plots']
                }
            }

            # Ensure metadata is JSON serializable
            def convert_to_serializable(obj):
                if isinstance(obj, (np.integer, np.floating)):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                elif isinstance(obj, datetime):
                    return obj.strftime('%Y-%m-%d %H:%M:%S')
                elif isinstance(obj, date):  # Add handling for date objects
                    return obj.strftime('%Y-%m-%d')
                elif pd.isnull(obj):  # Changed from isna to isnull
                    return None
                return obj

            # Process all entries recursively
            def process_dict(d):
                result = {}
                for k, v in d.items():
                    if isinstance(v, dict):
                        result[k] = process_dict(v)
                    elif isinstance(v, list):
                        result[k] = [
                            process_dict(item) if isinstance(item, dict)
                            else convert_to_serializable(item)
                            for item in v
                        ]
                    else:
                        result[k] = convert_to_serializable(v)
                return result

            # Process metadata
            print("\nProcessing metadata for saving...")
            serializable_metadata = process_dict(metadata)

            # Save to file with pretty printing
            if verbose:
                print("Processing metadata for saving...")
            with open(self.metadata_path, 'w') as f:
                json.dump(serializable_metadata, f, indent=4)
            if verbose:
                print(f"Metadata saved successfully to: {self.metadata_path}")

            # Save a backup copy with timestamp
            backup_path = os.path.join(
                os.path.dirname(self.metadata_path),
                f'metadata_backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
            )
            with open(backup_path, 'w') as f:
                json.dump(serializable_metadata, f, indent=4)
            if verbose:
                print(f"Metadata backup saved to: {backup_path}")

        except Exception as e:
            print(f"\nError saving metadata: {str(e)}")
            print("Metadata path:", self.metadata_path)
            print("Error details:", str(e))

            # Additional debugging information
            print("\nMetadata structure:")
            for key, value in metadata.items():
                print(f"{key}: {type(value)}")

            raise

    def _load_or_create_metadata(self):
        """Initialize or load existing metadata with all required fields."""
        if os.path.exists(self.metadata_path):
            try:
                with open(self.metadata_path, 'r') as f:
                    self.metadata = json.load(f)
                # Ensure all required fields exist even in loaded metadata
                self._ensure_metadata_structure()
            except Exception as e:
                print(f"Error loading metadata: {str(e)}. Creating new metadata.")
                self._create_new_metadata()
        else:
            self._create_new_metadata()

    def get_metadata_summary(self):
        """
        Get a summary of current pipeline metadata.

        Returns:
            dict: Summary of pipeline state and history
        """
        return {
            'creation_date': self.metadata['creation_date'],
            'data_count': len(self.metadata['data_dates']),
            'model_versions': len(self.metadata['model_versions']),
            'predictions_made': len(self.metadata['predictions']),
            'latest_data': self.model_dates['latest_data_date'],
            'last_training': self.model_dates['last_training_date'],
            'last_optimization': self.model_dates['last_optimization_date']
        }

    def fetch_earthquake_data(self, start_time=None, end_time=None, min_magnitude=2.5):
        """
        Fetch earthquake data from USGS API for a specific time period.

        Args:
            start_time (datetime): Start of time period
            end_time (datetime): End of time period
            min_magnitude (float): Minimum earthquake magnitude to include
        """
        try:
            # Construct the query URL for the USGS API
            base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

            # Format dates for the API
            if start_time is None:
                start_time = datetime.now() - timedelta(days=1)
            if end_time is None:
                end_time = datetime.now()

            params = {
                'format': 'geojson',
                'starttime': start_time.strftime('%Y-%m-%d'),
                'endtime': (end_time + timedelta(days=1)).strftime('%Y-%m-%d'),  # Add 1 day to include full end date
                'minmagnitude': min_magnitude,
                'orderby': 'time'
            }

            print(f"Fetching data from {params['starttime']} to {params['endtime']}")

            # Make the API request
            response = requests.get(base_url, params=params)
            response.raise_for_status()

            # Parse the JSON response
            data = response.json()
            earthquakes = data['features']

            processed_data = []
            for quake in earthquakes:
                properties = quake['properties']
                coordinates = quake['geometry']['coordinates']

                processed_data.append({
                    'time': datetime.fromtimestamp(properties['time'] / 1000),
                    'magnitude': properties['mag'],
                    'place': properties['place'],
                    'longitude': coordinates[0],
                    'latitude': coordinates[1],
                    'depth': coordinates[2],
                    'type': properties['type'],
                    'alert': properties.get('alert', 'none'),
                    'tsunami': properties['tsunami'],
                    'sig': properties['sig']
                })

            df = pd.DataFrame(processed_data)

            if len(df) > 0:
                print("\nData Collection Summary:")
                print("-" * 30)
                print(f"Total earthquakes collected: {len(df)}")
                print(f"Date range: {df['time'].min()} to {df['time'].max()}")
                print(f"Magnitude range: {df['magnitude'].min():.1f} to {df['magnitude'].max():.1f}")
                print("-" * 30)

            return df

        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

    def prepare_data(self, df, for_training=True, min_seq_length=1):
        """
        Prepare earthquake data for model training or prediction.

        Feature Selection:
        - magnitude: Primary indicator of earthquake strength
        - latitude/longitude: Geographic location
        - depth: Subsurface location
        - sig: USGS significance value combining magnitude/impact

        Sequence Construction:
        1. Group events by day
        2. Sort chronologically within each day
        3. Select up to max_earthquakes per day
        4. Create sliding windows of sequence_length days
        5. Generate target values from next day's data

        Scaling Methodology:
        - Features scaled using MinMaxScaler
        - Separate scalers for features and targets
        - Scaling ranges preserved for prediction

        Error Handling:
        - Minimum data threshold enforcement
        - Padding for undersized sequences
        - Invalid/missing value handling
        - Shape validation for tensor creation

        Args:
            df (pandas.DataFrame): Raw earthquake data
            for_training (bool): If True, prepare both features and targets
            min_seq_length (int): Minimum sequence length to use

        Returns:
            tuple: (sequence_tensor, target_tensor) for training
                  (sequence_tensor, None) for prediction

        Raises:
            ValueError: If insufficient data or invalid format
        """
        try:
            df = df.copy()

            # Ensure we have enough data
            if len(df) < 10:  # Minimum threshold for meaningful prediction
                print(f"Warning: Only {len(df)} events available. Need at least 10 for reliable prediction.")
                return None, None

            # Ensure time column is datetime
            df['time'] = pd.to_datetime(df['time'])
            # Create date column for grouping
            df['date'] = df['time'].dt.date

            # Sort by time to ensure proper sequence order
            df = df.sort_values('time')

            daily_groups = df.groupby('date')
            dates = sorted(df['date'].unique())

            print(f"Processing data from {dates[0]} to {dates[-1]}")

            # Ensure we have enough data for the sequence
            seq_length = min(len(dates) - 1, self.seq_length)
            seq_length = max(seq_length, min_seq_length)
            print(f"Using sequence length: {seq_length}")

            max_earthquakes = 60  # Fixed size for consistent tensors
            print(f"\nMaximum earthquakes per day: {max_earthquakes}")

            sequences = []
            targets = []

            # Create sequences and targets
            for i in range(len(dates) - seq_length):
                seq_dates = dates[i:i + seq_length]
                target_date = dates[i + seq_length]

                sequence = []
                for date in seq_dates:
                    # Get data for this date and select feature columns
                    day_data = daily_groups.get_group(date)[self.feature_columns].values

                    # Handle empty or undersized data
                    if len(day_data) == 0:
                        day_data = np.zeros((1, len(self.feature_columns)))

                    # Pad or truncate to max_earthquakes
                    if len(day_data) < max_earthquakes:
                        padding = np.zeros((max_earthquakes - len(day_data),
                                        len(self.feature_columns)))
                        day_data = np.vstack([day_data, padding])
                    else:
                        day_data = day_data[:max_earthquakes]

                    sequence.append(day_data)

                # Handle target data
                target_data = daily_groups.get_group(target_date)[self.target_columns].values
                if len(target_data) == 0:
                    target_data = np.zeros((1, len(self.target_columns)))

                if len(target_data) < max_earthquakes:
                    padding = np.zeros((max_earthquakes - len(target_data),
                                    len(self.target_columns)))
                    target_data = np.vstack([target_data, padding])
                else:
                    target_data = target_data[:max_earthquakes]

                sequences.append(sequence)
                targets.append(target_data)

            if not sequences:  # If no sequences could be created
                print("Warning: Could not create sequences from available data")
                return None, None

            # Convert to arrays and scale
            sequences = np.array(sequences)
            seq_shape = sequences.shape

            # Reshape for scaling
            sequences = sequences.reshape(-1, len(self.feature_columns))
            # Only fit scaler if we have non-zero data and are in training mode
            if sequences.any():  # Check if we have any non-zero values
                if for_training:
                    sequences = self.feature_scaler.fit_transform(sequences)
                else:
                    sequences = self.feature_scaler.transform(sequences)
            sequences = sequences.reshape(seq_shape)

            if not for_training:
                return torch.FloatTensor(sequences), None

            targets = np.array(targets)
            target_shape = targets.shape
            targets = targets.reshape(-1, len(self.target_columns))
            if targets.any():  # Check if we have any non-zero values
                if for_training:
                    targets = self.target_scaler.fit_transform(targets)
                else:
                    targets = self.target_scaler.transform(targets)
            targets = targets.reshape(target_shape)

            return torch.FloatTensor(sequences), torch.FloatTensor(targets)

        except Exception as e:
            print(f"Error in prepare_data: {str(e)}")
            print("DataFrame info:")
            print(df.info())
            return None, None

    def train_model(self, sequence_tensor, target_tensor, epochs=100, batch_size=32, learning_rate=0.001):
        """
        Train the transformer model on earthquake sequences.

        Training Process:
        1. Model Initialization
          - Create transformer architecture if none exists
          - Configure input/output dimensions
          - Set up attention heads

        2. Data Preparation
          - Create PyTorch datasets/dataloaders
          - Apply batch processing
          - Handle sequence padding

        3. Training Loop
          - Forward pass through transformer
          - Loss calculation (MSE + count prediction)
          - Backpropagation
          - Gradient updates
          - Performance tracking

        4. Optimization
          - Adam optimizer with learning rate scheduling
          - Early stopping on validation loss
          - Gradient clipping
          - Dropout regularization

        5. Checkpointing
          - Save best model states
          - Track training metrics
          - Store optimization parameters

        Error Handling:
        - Graceful handling of GPU memory issues
        - Batch failure recovery
        - Dimension mismatch detection
        - Invalid parameter checks

        Args:
            sequence_tensor (torch.Tensor): Shape (N, seq_len, max_eq, features)
            target_tensor (torch.Tensor): Shape (N, max_eq, target_features)
            epochs (int): Maximum training epochs
            batch_size (int): Samples per batch
            learning_rate (float): Initial learning rate

        Returns:
            list: Training history (loss values)

        Raises:
            RuntimeError: For training failures
            ValueError: For invalid inputs
        """
        try:
            if self.model is None:
                print("\nInitializing model...")
                input_dim = len(self.feature_columns)
                max_earthquakes = sequence_tensor.shape[2]
                num_heads = 4

                print(f"Input dimension: {input_dim}")
                print(f"Max earthquakes per day: {max_earthquakes}")
                print(f"Number of attention heads: {num_heads}")

                self.model = TransformerPredictor(
                    input_dim=input_dim,
                    hidden_dim=64,
                    num_layers=2,
                    num_heads=num_heads,
                    max_earthquakes=max_earthquakes
                )

            optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
            criterion = nn.MSELoss()

            print("\nPreparing dataset...")
            print(f"Sequence tensor shape: {sequence_tensor.shape}")
            print(f"Target tensor shape: {target_tensor.shape}")

            dataset = TensorDataset(sequence_tensor, target_tensor)
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            print("\nStarting training...")
            best_loss = float('inf')
            patience = 5
            patience_counter = 0
            training_history = []

            for epoch in range(epochs):
                self.model.train()
                total_loss = 0
                num_batches = 0

                for batch_sequences, batch_targets in dataloader:
                    optimizer.zero_grad()
                    try:
                        predictions, num_eq = self.model(batch_sequences)
                        loss = criterion(predictions, batch_targets)
                        loss.backward()
                        optimizer.step()
                        total_loss += loss.item()
                        num_batches += 1
                    except Exception as e:
                        print(f"Error in batch: {e}")
                        print(f"Batch shapes - Input: {batch_sequences.shape}, Target: {batch_targets.shape}")
                        continue

                avg_loss = total_loss / num_batches if num_batches > 0 else float('inf')
                training_history.append(avg_loss)

                if avg_loss < best_loss:
                    best_loss = avg_loss
                    patience_counter = 0
                    # Save best model
                    self.save_model_checkpoint(epoch, avg_loss)
                else:
                    patience_counter += 1

                if patience_counter >= patience:
                    print(f"\nEarly stopping at epoch {epoch+1}")
                    break

                if (epoch + 1) % 10 == 0:
                    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

            return training_history

        except Exception as e:
            print(f"\nError during training: {str(e)}")
            print("Tensor shapes:")
            print(f"Sequence tensor: {sequence_tensor.shape}")
            print(f"Target tensor: {target_tensor.shape}")
            raise

    def predict_next_day(self, recent_data):
        """
        Generate earthquake predictions for the next day.

        Uses the trained model to predict:
        - Number of earthquakes
        - Location (latitude, longitude)
        - Magnitude
        for each predicted earthquake

        Args:
            recent_data (pandas.DataFrame): Recent earthquake data for prediction

        Returns:
            pandas.DataFrame: Predicted earthquake parameters

        Example:
            >>> recent_data = pipeline.fetch_earthquake_data('day')
            >>> predictions = pipeline.predict_next_day(recent_data)
            >>> print(predictions)
        """
        self.model.eval()
        with torch.no_grad():
            try:
                # Prepare the input data
                sequence_tensor, _ = self.prepare_data(recent_data, for_training=False)

                # Generate predictions
                predictions, num_earthquakes_logits = self.model(sequence_tensor)
                predictions = predictions[0]  # Remove batch dimension

                # Calculate predicted number of earthquakes
                historical_mean = len(recent_data)
                predicted_prob = torch.sigmoid(num_earthquakes_logits[0]).item()
                predicted_count = int(round(predicted_prob * historical_mean))

                # Ensure reasonable bounds
                min_count = max(1, int(historical_mean * 0.5))  # At least 50% of historical
                max_count = min(int(historical_mean * 1.5), len(predictions))  # Can't be more than available predictions
                predicted_count = max(min_count, min(predicted_count, max_count))

                print(f"\nPrediction Details:")
                print(f"Historical average earthquakes: {historical_mean}")
                print(f"Predicted probability: {predicted_prob:.3f}")
                print(f"Available predictions: {len(predictions)}")
                print(f"Final predicted count: {predicted_count}")

                # Get confidence scores for each prediction
                confidence_scores = torch.norm(predictions, dim=1)

                # Select top k predictions based on confidence
                _, top_indices = torch.topk(confidence_scores, k=min(predicted_count, len(predictions)))
                selected_predictions = predictions[top_indices]

                # Convert to DataFrame
                pred_df = pd.DataFrame(
                    self.target_scaler.inverse_transform(selected_predictions),
                    columns=['predicted_magnitude', 'predicted_latitude', 'predicted_longitude']
                )

                return pred_df

            except Exception as e:
                print(f"Error during prediction: {str(e)}")
                raise

    def evaluate_predictions(self, predictions, actual_data):
        """
        Evaluate prediction accuracy using multiple metrics.

        Calculates:
        - Location error (km)
        - Magnitude error
        - False positives/negatives
        - Precision, recall, F1 score

        Args:
            predictions (DataFrame): Predicted earthquakes
            actual_data (DataFrame): Actual earthquake data

        Returns:
            dict: Evaluation metrics

        Example:
            >>> predictions = pipeline.predict_next_day(recent_data)
            >>> next_day_data = pipeline.fetch_earthquake_data('day')
            >>> metrics = pipeline.evaluate_predictions(predictions, next_day_data)
            >>> print(f"Average location error: {metrics['avg_location_error']:.2f} km")
        """
        def haversine_distance(lat1, lon1, lat2, lon2):
            """Calculate distance between two points in km."""
            R = 6371  # Earth's radius in km

            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1

            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            c = 2 * np.arcsin(np.sqrt(a))
            return R * c

        try:
            # Initialize metrics dictionary
            metrics = {
                'location_errors': [],
                'magnitude_errors': [],
                'false_positives': 0,
                'false_negatives': 0,
                'correct_predictions': 0,
                'avg_location_error': 0,
                'avg_magnitude_error': 0,
                'precision': 0,
                'recall': 0,
                'f1_score': 0
        }

            # Check if we have actual data to compare against
            if actual_data is None or actual_data.empty:
                print("No actual data available for evaluation")
                return metrics

            # Ensure actual data has required columns
            required_columns = ['magnitude', 'latitude', 'longitude']
            if not all(col in actual_data.columns for col in required_columns):
                print("Actual data missing required columns")
                print("Available columns:", actual_data.columns)
                print("Required columns:", required_columns)
                return {
                    'location_errors': [],
                    'magnitude_errors': [],
                    'false_positives': len(predictions),
                    'false_negatives': 0,
                    'correct_predictions': 0,
                    'avg_location_error': 0,
                    'avg_magnitude_error': 0,
                    'precision': 0,
                    'recall': 0,
                    'f1_score': 0
                }

            # Get actual events
            actual_events = actual_data[self.target_columns].values
            pred_events = predictions[['predicted_magnitude', 'predicted_latitude',
                                    'predicted_longitude']].values

            print("\nEvaluation Summary:")
            print(f"Number of predicted events: {len(pred_events)}")
            print(f"Number of actual events: {len(actual_events)}")

            # Track matched events
            matched_actual_indices = set()

            # Evaluate each prediction
            for pred_idx, pred_event in enumerate(pred_events):
                min_dist = float('inf')
                best_match_idx = None

                # Find closest actual earthquake
                for actual_idx, actual_event in enumerate(actual_events):
                    if actual_idx in matched_actual_indices:
                        continue

                    dist = haversine_distance(
                        pred_event[1], pred_event[2],
                        actual_event[1], actual_event[2]
                    )

                    if dist < min_dist:
                        min_dist = dist
                        best_match_idx = actual_idx

                # Process match if found within 100km
                if min_dist < 100 and best_match_idx is not None:
                    metrics['correct_predictions'] += 1
                    metrics['location_errors'].append(min_dist)

                    mag_error = abs(pred_event[0] - actual_events[best_match_idx][0])
                    metrics['magnitude_errors'].append(mag_error)

                    matched_actual_indices.add(best_match_idx)
                else:
                    metrics['false_positives'] += 1

            # Count unmatched actual events
            metrics['false_negatives'] = len(actual_events) - len(matched_actual_indices)

            # Calculate averages and scores
            metrics['avg_location_error'] = np.mean(metrics['location_errors']) if metrics['location_errors'] else 0
            metrics['avg_magnitude_error'] = np.mean(metrics['magnitude_errors']) if metrics['magnitude_errors'] else 0

            # Calculate precision, recall, F1
            if metrics['correct_predictions'] + metrics['false_positives'] > 0:
                metrics['precision'] = metrics['correct_predictions'] / (metrics['correct_predictions'] + metrics['false_positives'])
            else:
                metrics['precision'] = 0

            if metrics['correct_predictions'] + metrics['false_negatives'] > 0:
                metrics['recall'] = metrics['correct_predictions'] / (metrics['correct_predictions'] + metrics['false_negatives'])
            else:
                metrics['recall'] = 0

            if metrics['precision'] + metrics['recall'] > 0:
                metrics['f1_score'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])
            else:
                metrics['f1_score'] = 0

            # Print detailed results
            print("\nDetailed Metrics:")
            print(f"Correct Predictions: {metrics['correct_predictions']}")
            print(f"False Positives: {metrics['false_positives']}")
            print(f"False Negatives: {metrics['false_negatives']}")
            print(f"Average Location Error: {metrics['avg_location_error']:.2f} km")
            print(f"Average Magnitude Error: {metrics['avg_magnitude_error']:.2f}")
            print(f"Precision: {metrics['precision']:.3f}")
            print(f"Recall: {metrics['recall']:.3f}")
            print(f"F1 Score: {metrics['f1_score']:.3f}")

            return metrics

        except Exception as e:
            print(f"Error during evaluation: {str(e)}")
            print(f"Predictions shape: {predictions.shape}")
            print(f"Actual data shape: {actual_data.shape if hasattr(actual_data, 'shape') else (0,0)}")
            raise

    def optimize_model(self, new_data, current_metrics, learning_rate=0.0001):
        """
        Fine-tune model on new earthquake data.

        Optimization Strategy:
        1. Data Integration
          - Process new earthquake events
          - Create sequence tensors
          - Apply consistent scaling

        2. Loss Analysis
          - Calculate baseline performance
          - Identify prediction weaknesses
          - Weight loss components

        3. Model Update
          - Incremental learning on new data
          - Maintain historical knowledge
          - Adaptive learning rates
          - Gradient accumulation

        4. Performance Monitoring
          - Track metrics before/after
          - Validate improvements
          - Store optimization history
          - Update metadata

        5. Model Management
          - Save optimized state
          - Update checkpoint information
          - Clean old checkpoints
          - Log changes

        Optimization Parameters:
        - learning_rate: Small for stable updates
        - loss_weights: Balanced for all targets
        - gradient_clip: Prevent extreme updates
        - update_steps: Multiple passes if needed

        Args:
            new_data (DataFrame): New earthquake events
            current_metrics (dict): Current performance metrics
            learning_rate (float): Fine-tuning rate

        Raises:
            ValueError: For invalid optimization attempts
            RuntimeError: For optimization failures
        """
        try:
            print("\nOptimizing model...")

            if len(new_data) == 0:
                print("No data available for optimization")
                return

            # Create sequence using the new day's data
            sequence_data = new_data.copy()
            sequence_data['date'] = pd.to_datetime(sequence_data['time']).dt.date

            dates = sorted(sequence_data['date'].unique())
            print(f"Optimizing with data from: {dates[0]}")

            max_earthquakes = 60  # Match training dimensions

            # Prepare day's data with padding
            day_data = sequence_data[self.feature_columns].values
            if len(day_data) < max_earthquakes:
                padding = np.zeros((max_earthquakes - len(day_data), len(self.feature_columns)))
                day_data = np.vstack([day_data, padding])
            elif len(day_data) > max_earthquakes:
                day_data = day_data[:max_earthquakes]

            # Scale features
            scaled_features = self.feature_scaler.transform(day_data)
            sequence_tensor = torch.FloatTensor(scaled_features).unsqueeze(0).unsqueeze(0)

            # Prepare target data
            target_data = sequence_data[self.target_columns].values
            if len(target_data) < max_earthquakes:
                padding = np.zeros((max_earthquakes - len(target_data), len(self.target_columns)))
                target_data = np.vstack([target_data, padding])
            elif len(target_data) > max_earthquakes:
                target_data = target_data[:max_earthquakes]

            scaled_targets = self.target_scaler.transform(target_data)
            target_tensor = torch.FloatTensor(scaled_targets).unsqueeze(0)

            print(f"Optimization tensors - Input: {sequence_tensor.shape}, Target: {target_tensor.shape}")

            # Perform optimization
            optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
            criterion = nn.MSELoss()

            self.model.train()
            optimizer.zero_grad()

            predictions, num_eq = self.model(sequence_tensor)
            loss = criterion(predictions.squeeze(0), target_tensor.squeeze(0))
            loss.backward()
            optimizer.step()

            print(f"Optimization step completed. Loss: {loss.item():.4f}")

            # Evaluate improvement
            self.model.eval()
            with torch.no_grad():
                new_predictions, _ = self.model(sequence_tensor)
                new_loss = criterion(new_predictions.squeeze(0), target_tensor.squeeze(0))

            print(f"Post-optimization loss: {new_loss.item():.4f}")

            # Update metadata
            self.model_dates['last_optimization_date'] = dates[0]
            self._save_metadata()

        except Exception as e:
            print(f"Error during optimization: {str(e)}")
            print("\nError details:")
            print(f"New data shape: {new_data.shape if hasattr(new_data, 'shape') else len(new_data)}")
            print("Metrics:", current_metrics)
            raise

    def save_model_checkpoint(self, epoch, loss, metrics=None):
        """
        Save model checkpoint with comprehensive metadata.

        Saves:
        - Model state
        - Scalers
        - Training metrics
        - Date information

        Args:
            epoch (int): Current training epoch
            loss (float): Current loss value
            metrics (dict, optional): Additional metrics to save

        Example:
            >>> pipeline.save_model_checkpoint(epoch=50, loss=0.123, metrics={'f1': 0.85})
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Create dated directory structure
        if self.model_dates['latest_data_date']:
            model_dir = os.path.join(self.dirs['models'],
                                   self.model_dates['latest_data_date'][:7])
            os.makedirs(model_dir, exist_ok=True)
        else:
            model_dir = self.dirs['models']

        # Save model state
        model_path = os.path.join(model_dir, f'model_checkpoint_{timestamp}.pth')
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'feature_scaler': self.feature_scaler,
            'target_scaler': self.target_scaler,
            'loss': loss,
            'metrics': metrics,
            'timestamp': timestamp,
            'model_dates': self.model_dates.copy()
        }

        torch.save(checkpoint, model_path)

        # Save configuration
        config_path = os.path.join(model_dir, f'model_config_{timestamp}.json')
        config = {
            'timestamp': timestamp,
            'epoch': epoch,
            'loss': float(loss),
            'sequence_length': self.seq_length,
            'feature_columns': self.feature_columns,
            'target_columns': self.target_columns,
            'model_dates': self.model_dates,
            'metrics': metrics
        }

        with open(config_path, 'w') as f:
            json.dump(config, f, indent=4)

        # Update metadata
        self.metadata['model_versions'].append({
            'timestamp': timestamp,
            'path': model_path,
            'config_path': config_path,
            'metrics': metrics
        })
        self._save_metadata()

        print(f"Model checkpoint saved: {model_path}")
        print(f"Configuration saved: {config_path}")

    def load_latest_model(self):
        """
        Load the most recent model checkpoint.

        Handles:
        - Finding the latest checkpoint
        - Loading model state
        - Restoring scalers
        - Updating metadata

        Returns:
            bool: True if model loaded successfully, False otherwise

        Example:
            >>> success = pipeline.load_latest_model()
            >>> if success:
            ...     print("Model loaded successfully")
        """
        try:
            # Find all model checkpoints
            checkpoint_pattern = os.path.join(self.dirs['models'], '**',
                                           'model_checkpoint_*.pth')
            model_files = glob.glob(checkpoint_pattern, recursive=True)

            if not model_files:
                print("No saved models found")
                return False

            # Get the latest checkpoint
            latest_model = max(model_files, key=os.path.getctime)
            checkpoint = torch.load(latest_model)

            # Load model state
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.feature_scaler = checkpoint['feature_scaler']
            self.target_scaler = checkpoint['target_scaler']

            # Update model dates
            if 'model_dates' in checkpoint:
                self.model_dates.update(checkpoint['model_dates'])

            print(f"Loaded model from: {latest_model}")
            print(f"Checkpoint epoch: {checkpoint['epoch']}")
            print(f"Loss: {checkpoint['loss']}")

            if 'metrics' in checkpoint and checkpoint['metrics']:
                print("\nMetrics at checkpoint:")
                for metric, value in checkpoint['metrics'].items():
                    print(f"{metric}: {value}")

            return True

        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return False

    def save_daily_data(self, df, date=None):
        """
        Save daily earthquake data with comprehensive metadata.

        Args:
            df (DataFrame): Earthquake data to save
            date (str, optional): Specific date for the data. Defaults to current date.

        Example:
            >>> data = pipeline.fetch_earthquake_data('day')
            >>> pipeline.save_daily_data(data, '2024-11-18')
        """
        if date is None:
            date = datetime.now().strftime('%Y-%m-%d')

        # Create dated directory structure
        year_month = date[:7]  # YYYY-MM
        data_dir = os.path.join(self.dirs['data'], year_month)
        os.makedirs(data_dir, exist_ok=True)

        # Save data
        filename = f'earthquake_data_{date}.csv'
        filepath = os.path.join(data_dir, filename)
        df.to_csv(filepath, index=False)

        # Save daily summary
        summary_path = os.path.join(data_dir, f'summary_{date}.json')
        summary = {
            'date': date,
            'total_earthquakes': len(df),
            'magnitude_range': {
                'min': float(df['magnitude'].min()),
                'max': float(df['magnitude'].max()),
                'mean': float(df['magnitude'].mean())
            },
            'location_bounds': {
                'lat': {'min': float(df['latitude'].min()),
                       'max': float(df['latitude'].max())},
                'lon': {'min': float(df['longitude'].min()),
                       'max': float(df['longitude'].max())}
            },
            'depth_stats': {
                'min': float(df['depth'].min()),
                'max': float(df['depth'].max()),
                'mean': float(df['depth'].mean())
            },
            'saved_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=4)

        # Update metadata
        self.metadata['data_dates'].append({
            'date': date,
            'filepath': filepath,
            'summary_path': summary_path,
            'stats': summary
        })
        self._save_metadata(verbose=True)

        # Update pipeline date tracking
        self.model_dates['latest_data_date'] = date

        print(f"Data saved: {filepath}")
        print(f"Summary saved: {summary_path}")

    def load_daily_data(self, date_str):
        """
        Load earthquake data for a specific date from saved files.

        Args:
            date_str (str): Date in 'YYYY-MM-DD' format

        Returns:
            pandas.DataFrame: DataFrame containing earthquake data for the specified date
        """
        try:
            # Construct the file path
            year_month = date_str[:7]  # YYYY-MM
            filename = f'earthquake_data_{date_str}.csv'
            filepath = os.path.join(self.dirs['data'], year_month, filename)

            if os.path.exists(filepath):
                # Load the data
                df = pd.read_csv(filepath)

                # Convert time column back to datetime
                df['time'] = pd.to_datetime(df['time'])

                print(f"Loaded data for {date_str}: {len(df)} earthquakes")
                return df
            else:
                print(f"No data file found for {date_str}")
                return None

        except Exception as e:
            print(f"Error loading data for {date_str}: {str(e)}")
            return None

    def save_predictions(self, predictions, prediction_date, actual_data=None):
        """
        Save predictions and optionally actual data for comparison.

        Args:
            predictions (DataFrame): Predicted earthquake parameters
            prediction_date (str): Date these predictions are for (YYYY-MM-DD)
            actual_data (DataFrame, optional): Actual earthquake data if available
        """
        try:
            # Create prediction directory structure
            year_month = prediction_date[:7]  # YYYY-MM
            pred_dir = os.path.join(self.dirs['predictions'], year_month)
            os.makedirs(pred_dir, exist_ok=True)

            # Save predictions
            pred_filename = f'predictions_{prediction_date}.csv'
            pred_filepath = os.path.join(pred_dir, pred_filename)
            predictions.to_csv(pred_filepath, index=False)
            print(f"\nPredictions saved to: {pred_filepath}")

            # Save comparison if actual data is available
            if actual_data is not None:
                comparison = {
                    'date': prediction_date,
                    'num_predicted': len(predictions),
                    'num_actual': len(actual_data),
                    'prediction_stats': {
                        'magnitude': {
                            'mean': float(predictions['predicted_magnitude'].mean()),
                            'min': float(predictions['predicted_magnitude'].min()),
                            'max': float(predictions['predicted_magnitude'].max())
                        },
                        'location': {
                            'lat_range': [
                                float(predictions['predicted_latitude'].min()),
                                float(predictions['predicted_latitude'].max())
                            ],
                            'lon_range': [
                                float(predictions['predicted_longitude'].min()),
                                float(predictions['predicted_longitude'].max())
                            ]
                        }
                    },
                    'actual_stats': {
                        'magnitude': {
                            'mean': float(actual_data['magnitude'].mean()),
                            'min': float(actual_data['magnitude'].min()),
                            'max': float(actual_data['magnitude'].max())
                        },
                        'location': {
                            'lat_range': [
                                float(actual_data['latitude'].min()),
                                float(actual_data['latitude'].max())
                            ],
                            'lon_range': [
                                float(actual_data['longitude'].min()),
                                float(actual_data['longitude'].max())
                            ]
                        }
                    },
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }

                # Save comparison summary
                summary_filename = f'comparison_{prediction_date}.json'
                summary_filepath = os.path.join(pred_dir, summary_filename)
                with open(summary_filepath, 'w') as f:
                    json.dump(comparison, f, indent=4)
                print(f"Comparison summary saved to: {summary_filepath}")

                # Update performance history
                self.performance_history.append({
                    'date': prediction_date,
                    'metrics': self.evaluate_predictions(predictions, actual_data),
                    'comparison': comparison
                })

        except Exception as e:
            print(f"\nError saving predictions: {str(e)}")
            print(f"Prediction date: {prediction_date}")
            print(f"Number of predictions: {len(predictions)}")
            if actual_data is not None:
                print(f"Number of actual events: {len(actual_data)}")
            raise

    def load_predictions(self, prediction_date):
        """
        Load saved predictions for a specific date.

        Args:
            prediction_date (str): Date to load predictions for (YYYY-MM-DD)

        Returns:
            DataFrame: Loaded predictions, or None if not found
        """
        try:
            # Construct file path
            year_month = prediction_date[:7]  # YYYY-MM
            pred_dir = os.path.join(self.dirs['predictions'], year_month)
            pred_filepath = os.path.join(pred_dir, f'predictions_{prediction_date}.csv')

            if os.path.exists(pred_filepath):
                predictions = pd.read_csv(pred_filepath)
                print(f"\nLoaded predictions for {prediction_date}")
                print(f"Number of predictions: {len(predictions)}")
                return predictions
            else:
                print(f"\nNo predictions found for {prediction_date}")
                return None

        except Exception as e:
            print(f"\nError loading predictions: {str(e)}")
            print(f"Attempted to load from: {pred_filepath}")
            return None

    def plot_performance_history(self, save=True):
        """
        Create comprehensive visualization of prediction performance metrics.
        """
        if not self.performance_history:
            print("No performance history available")
            return

        try:
            # Extract data for plotting
            dates = []
            location_errors = []
            magnitude_errors = []
            correct_predictions = []
            false_positives = []
            false_negatives = []
            precision_scores = []
            recall_scores = []
            f1_scores = []

            for entry in self.performance_history:
                # Convert string date to datetime if needed
                if isinstance(entry['date'], str):
                    plot_date = datetime.strptime(entry['date'], '%Y-%m-%d')
                else:
                    plot_date = entry['date']
                dates.append(plot_date)

                metrics = entry['metrics']
                location_errors.append(metrics.get('avg_location_error', 0))
                magnitude_errors.append(metrics.get('avg_magnitude_error', 0))
                correct_predictions.append(metrics.get('correct_predictions', 0))
                false_positives.append(metrics.get('false_positives', 0))
                false_negatives.append(metrics.get('false_negatives', 0))
                precision_scores.append(metrics.get('precision', 0))
                recall_scores.append(metrics.get('recall', 0))
                f1_scores.append(metrics.get('f1_score', 0))

            # Create figure with subplots
            plt.figure(figsize=(20, 15))

            # 1. Error Metrics
            plt.subplot(3, 2, 1)
            plt.plot(dates, location_errors, marker='o', label='Location Error (km)',
                    color='blue')
            plt.title('Location Error Over Time')
            plt.xlabel('Date')
            plt.ylabel('Average Error (km)')
            plt.grid(True)
            plt.xticks(rotation=45)
            plt.legend()

            # 2. Magnitude Error
            plt.subplot(3, 2, 2)
            plt.plot(dates, magnitude_errors, marker='o', label='Magnitude Error',
                    color='red')
            plt.title('Magnitude Error Over Time')
            plt.xlabel('Date')
            plt.ylabel('Average Error')
            plt.grid(True)
            plt.xticks(rotation=45)
            plt.legend()

            # 3. Prediction Counts
            plt.subplot(3, 2, 3)
            x = np.arange(len(dates))
            width = 0.25
            plt.bar(x - width, correct_predictions, width, label='Correct',
                  color='green')
            plt.bar(x, false_positives, width, label='False Positives',
                  color='red', alpha=0.7)
            plt.bar(x + width, false_negatives, width, label='False Negatives',
                  color='orange', alpha=0.7)
            plt.title('Prediction Breakdown Over Time')
            plt.xlabel('Date')
            plt.ylabel('Count')
            # Use datetime objects for date formatting
            plt.xticks(x, [d.strftime('%Y-%m-%d') for d in dates], rotation=45)
            plt.legend()
            plt.grid(True)

            # 4. Precision-Recall
            plt.subplot(3, 2, 4)
            plt.plot(dates, precision_scores, marker='o', label='Precision',
                    color='purple')
            plt.plot(dates, recall_scores, marker='s', label='Recall',
                    color='green')
            plt.title('Precision and Recall Over Time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.grid(True)
            plt.xticks(rotation=45)
            plt.legend()

            # 5. F1 Score
            plt.subplot(3, 2, 5)
            plt.plot(dates, f1_scores, marker='o', label='F1 Score',
                    color='blue')
            plt.fill_between(dates, 0, f1_scores, alpha=0.2, color='blue')
            plt.title('F1 Score Over Time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.grid(True)
            plt.xticks(rotation=45)
            plt.legend()

            # 6. Summary Statistics
            plt.subplot(3, 2, 6)
            summary_stats = {
                'Avg Location Error': np.mean(location_errors),
                'Avg Magnitude Error': np.mean(magnitude_errors),
                'Avg Precision': np.mean(precision_scores),
                'Avg Recall': np.mean(recall_scores),
                'Avg F1 Score': np.mean(f1_scores)
            }

            y_pos = np.arange(len(summary_stats))
            plt.barh(y_pos, list(summary_stats.values()))
            plt.yticks(y_pos, list(summary_stats.keys()))
            plt.title('Overall Performance Summary')
            plt.xlabel('Average Value')

            plt.tight_layout()

            if save:
                # Save plots with timestamp
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                plot_path = os.path.join(self.dirs['plots'],
                                      f'performance_history_{timestamp}.png')
                plt.savefig(plot_path, dpi=300, bbox_inches='tight')

                # Save summary report
                report_path = os.path.join(self.dirs['plots'],
                                        f'performance_summary_{timestamp}.txt')
                with open(report_path, 'w') as f:
                    f.write("Performance Summary\n")
                    f.write("==================\n\n")
                    for metric, value in summary_stats.items():
                        f.write(f"{metric}: {value:.4f}\n")
                    f.write("\nDetailed Metrics Available in Plot\n")

                plt.close()

                print(f"\nPerformance plot saved: {plot_path}")
                print(f"Summary report saved: {report_path}")

            # Print summary to console
            print("\nPerformance Summary:")
            print("===================")
            for metric, value in summary_stats.items():
                print(f"{metric}: {value:.4f}")

        except Exception as e:
            print(f"Error creating performance plots: {str(e)}")
            # Additional debugging information
            if self.performance_history:
                print("\nFirst entry in performance history:")
                print(f"Date type: {type(self.performance_history[0]['date'])}")
                print(f"Date value: {self.performance_history[0]['date']}")
            raise

    def run_pipeline(self, days_to_process=30, continuous=False, update_interval=3600):
        """
        Execute the complete earthquake prediction pipeline.
        """
        try:
            print("\nInitializing Earthquake Prediction Pipeline")
            print("==========================================")

            # Initialize dates
            current_time = datetime.now()
            end_date = current_time - timedelta(days=1)  # Process until yesterday
            start_date = end_date - timedelta(days=days_to_process)
            print(f"\nProcessing historical period: {start_date.date()} to {end_date.date()}")

            # First collect initial training data (7 days)
            initial_end = start_date + timedelta(days=7)
            print(f"\nCollecting initial training data: {start_date.date()} to {initial_end.date()}")

            # Fetch and store initial training data
            daily_data = []
            current = start_date
            while current <= initial_end:
                data = self.fetch_earthquake_data(
                    start_time=current,
                    min_magnitude=2.5
                )
                if data is not None and len(data) >= 10:  # Ensure minimum data threshold
                    self.save_daily_data(data, current.strftime('%Y-%m-%d'))
                    daily_data.append(data)
                current += timedelta(days=1)

            # Train initial model if we have enough data
            if len(daily_data) >= self.seq_length:
                print("\nPreparing initial training data...")
                training_data = pd.concat(daily_data)
                sequence_tensor, target_tensor = self.prepare_data(training_data)

                if sequence_tensor is not None and target_tensor is not None:
                    print("\nTraining initial model...")
                    self.train_model(sequence_tensor, target_tensor)
                    self.model_dates['last_training_date'] = initial_end.strftime('%Y-%m-%d')
                else:
                    raise ValueError("Could not prepare training data")
            else:
                raise ValueError("Insufficient data for initial training")

            # Process remaining historical days
            current = initial_end + timedelta(days=1)
            while current <= end_date:
                print(f"\nProcessing historical day: {current.date()}")

                data = self.fetch_earthquake_data(
                    start_time=current,
                    min_magnitude=2.5
                )

                if data is not None and len(data) >= 10:
                    self.save_daily_data(data, current.strftime('%Y-%m-%d'))

                    # Generate and evaluate predictions
                    try:
                        sequence_tensor, _ = self.prepare_data(data, for_training=False)
                        if sequence_tensor is not None:
                            predictions = self.predict_next_day(data)
                            if predictions is not None:
                                next_day = current + timedelta(days=1)
                                self.save_predictions(predictions, next_day.strftime('%Y-%m-%d'))

                                # If we have data for the next day, evaluate predictions
                                if next_day <= end_date:
                                    next_day_data = self.fetch_earthquake_data(
                                        start_time=next_day,
                                        min_magnitude=2.5
                                    )
                                    if next_day_data is not None and len(next_day_data) >= 10:
                                        metrics = self.evaluate_predictions(predictions, next_day_data)
                                        self.optimize_model(next_day_data, metrics)
                    except Exception as e:
                        print(f"Error processing day {current.date()}: {str(e)}")
                        continue

                current += timedelta(days=1)

            print("\nHistorical data processing completed")

            # Switch to continuous monitoring if requested
            if continuous:
                print("\nSwitching to continuous monitoring mode...")

                # Wait until we have a complete day of data
                current_time = datetime.now()
                if current_time.hour < 23:  # If it's not near the end of the day
                    next_processing_time = datetime(
                        current_time.year,
                        current_time.month,
                        current_time.day,
                        23, 0  # Set to 11 PM
                    )

                    wait_seconds = (next_processing_time - current_time).total_seconds()
                    print(f"\nCurrent time: {current_time}")
                    print(f"Waiting until {next_processing_time} to begin continuous monitoring...")
                    print(f"Will start continuous monitoring in {wait_seconds/3600:.1f} hours")

                    time.sleep(wait_seconds)

                # Start continuous monitoring
                self.run_continuous_monitoring(update_interval=update_interval)

        except Exception as e:
            print(f"\nError in pipeline execution: {str(e)}")
            print(f"Error details: {str(e)}")
            raise

    def run_continuous_monitoring(self, update_interval=3600):
        """Run continuous monitoring of earthquakes and predictions."""
        try:
            print("\nStarting continuous monitoring...")

            while True:
                current_time = datetime.now()

                # Only process complete days
                if current_time.hour < 23:
                    wait_seconds = (datetime(
                        current_time.year,
                        current_time.month,
                        current_time.day,
                        23, 0
                    ) - current_time).total_seconds()

                    print(f"\nWaiting for complete day data...")
                    print(f"Next processing time: {current_time + timedelta(seconds=wait_seconds)}")
                    time.sleep(wait_seconds)
                    continue

                # Process the previous complete day
                process_date = current_time - timedelta(days=1)
                print(f"\nProcessing data for: {process_date.date()}")

                data = self.fetch_earthquake_data(
                    start_time=process_date,
                    min_magnitude=2.5
                )

                if data is not None and len(data) >= 10:
                    try:
                        predictions = self.predict_next_day(data)
                        if predictions is not None:
                            self.save_predictions(
                                predictions,
                                current_time.strftime('%Y-%m-%d')
                            )
                            print("Predictions generated and saved successfully")
                    except Exception as e:
                        print(f"Error during prediction: {str(e)}")

                # Wait for next update interval
                print(f"\nWaiting {update_interval} seconds until next check...")
                time.sleep(update_interval)

        except KeyboardInterrupt:
            print("\nContinuous monitoring stopped by user")
        except Exception as e:
            print(f"\nError in continuous monitoring: {str(e)}")
            raise

In [ ]:
# First, mount Google Drive (if using Colab)
from google.colab import drive
drive.mount('/content/drive')

# Set up base directory in Google Drive
base_path = '/content/drive/My Drive/earthquake_data'

# Initialize the pipeline
pipeline = EarthquakePipeline(drive_path=base_path)

# Run historical processing then switch to continuous
pipeline.run_pipeline(days_to_process=30, continuous=True, update_interval=3600)

# OR just run continuous monitoring with existing model
# pipeline.run_continuous_monitoring(update_interval=3600)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing metadata for saving...

Initializing Earthquake Prediction Pipeline

Processing historical period: 2024-10-19 to 2024-11-18

Fetching data from 2024-10-19 to 2024-11-20

Data Collection Summary:
------------------------------
Total earthquakes collected: 1247
Date range: 2024-10-19 00:18:53.783000 to 2024-11-19 12:56:02.383000
Magnitude range: 2.5 to 6.8
------------------------------

Processing metadata for saving...
Processing metadata for saving...
Metadata saved successfully to: /content/drive/My Drive/earthquake_data/pipeline_metadata.json
Metadata backup saved to: /content/drive/My Drive/earthquake_data/metadata_backup_20241119_133609.json
Data saved: /content/drive/My Drive/earthquake_data/data/2024-10/earthquake_data_2024-10-19.csv
Summary saved: /content/drive/My Drive/earthquake_data/data/2024-10/summary_2024-10-19.json
Fetching data fr